## Marriage Trens in India Project

In [29]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('marriage_data_india.csv')
display(data.head())

,ID,Marriage_Type,Age_at_Marriage,Gender,Education_Level,Caste_Match,Religion,Parental_Approval,Urban_Rural,Dowry_Exchanged,Marital_Satisfaction,Divorce_Status,Children_Count,Income_Level,Years_Since_Marriage,Spouse_Working,Inter-Caste,Inter-Religion
0,1,Love,23,Male,Graduate,Different,Hindu,No,Urban,No,Medium,Yes,5,Middle,34,No,No,No
1,2,Love,28,Female,School,Same,Hindu,Yes,Rural,Yes,Low,No,3,Middle,42,No,No,Yes
2,3,Arranged,39,Male,Postgraduate,Same,Muslim,Yes,Rural,No,Medium,No,0,High,25,No,No,No
3,4,Arranged,26,Female,School,Different,Hindu,Yes,Urban,Yes,Low,No,0,High,12,No,Yes,No
4,5,Love,32,Female,Graduate,Same,Hindu,Partial,Rural,Yes,Medium,No,1,Middle,41,No,No,Yes


In [30]:
display(data.isna().sum())

ID                      0
Marriage_Type           0
Age_at_Marriage         0
Gender                  0
Education_Level         0
Caste_Match             0
Religion                0
Parental_Approval       0
Urban_Rural             0
Dowry_Exchanged         0
Marital_Satisfaction    0
Divorce_Status          0
Children_Count          0
Income_Level            0
Years_Since_Marriage    0
Spouse_Working          0
Inter-Caste             0
Inter-Religion          0
dtype: int64

In [31]:
X = data.drop(columns=['ID','Divorce_Status'])
y = data['Divorce_Status']

X['Inter_Caste'] = X['Inter-Caste']
X['Inter_Religion'] = X['Inter-Religion']
X.drop(columns=['Inter-Caste','Inter-Religion'], inplace=True)

In [32]:
categorical_cols = [
    'Marriage_Type', 
    'Gender', 
    'Education_Level', 
    'Caste_Match', 
    'Religion', 
    'Parental_Approval', 
    'Urban_Rural', 
    'Dowry_Exchanged',
    'Marital_Satisfaction', 
    'Income_Level',
    'Spouse_Working', 
    'Inter_Caste', 
    'Inter_Religion']

X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
display(X.head())

,Age_at_Marriage,Children_Count,Years_Since_Marriage,Marriage_Type_Love,Gender_Male,Education_Level_PhD,Education_Level_Postgraduate,Education_Level_School,Caste_Match_Same,Religion_Hindu,...,Urban_Rural_Urban,Dowry_Exchanged_Not Disclosed,Dowry_Exchanged_Yes,Marital_Satisfaction_Low,Marital_Satisfaction_Medium,Income_Level_Low,Income_Level_Middle,Spouse_Working_Yes,Inter_Caste_Yes,Inter_Religion_Yes
0,23,5,34,True,True,False,False,False,False,True,...,True,False,False,False,True,False,True,False,False,False
1,28,3,42,True,False,False,False,True,True,True,...,False,False,True,True,False,False,True,False,False,True
2,39,0,25,False,True,False,True,False,True,False,...,False,False,False,False,True,False,False,False,False,False
3,26,0,12,False,False,False,False,True,False,True,...,True,False,True,True,False,False,False,False,True,False
4,32,1,41,True,False,False,False,False,True,True,...,False,False,True,False,True,False,True,False,False,True


In [33]:
le = LabelEncoder()
y = le.fit_transform(y)

In [59]:
from imblearn.over_sampling import SMOTE
obj = SMOTE()
X_new, y_new = obj.fit_resample(X, y)

In [67]:
import statsmodels.api as sm
# def backwardElimination(x, y, sl):
#     numVars = len(x[0])
#     for i in range(0, numVars):
#         obj_OLS = sm.OLS(y, x).fit()
#         display(obj_OLS.summary())
#         maxVar = max(obj_OLS.pvalues).astype(float)
#         if maxVar > sl:
#             for j in range(0, numVars - i):
#                 if (obj_OLS.pvalues[j].astype(float) == maxVar):
#                     x = np.delete(x, j, 1)
#     obj_OLS.summary()
#     return x

def backwardElimination(x, y, sl):
    numVars = len(x[0])
    indices = list(range(numVars))
    for i in range(0, numVars):
        obj_OLS = sm.OLS(y, x).fit()
        maxVar = max(obj_OLS.pvalues).astype(float)
        if maxVar > sl:
            for j in range(0, len(indices)):
                if (obj_OLS.pvalues[j].astype(float) == maxVar):
                    x = np.delete(x, j, 1)
                    indices.pop(j)
                    break
    display(obj_OLS.summary())
    return x, indices      
   
      
SL = 0.05
X_backe = np.append(arr=np.ones((len(X_new),1)), values=X_new, axis=1)
X_backe = X_backe.astype('float64')
X_sig = X_backe
X_Modeled, indices = backwardElimination(X_sig, y_new, SL)
display(X_Modeled)
display(indices)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.479
Model:                            OLS   Adj. R-squared:                  0.478
Method:                 Least Squares   F-statistic:                     660.6
Date:                Mon, 14 Apr 2025   Prob (F-statistic):               0.00
Time:                        12:09:21   Log-Likelihood:                -7197.4
No. Observations:               17998   AIC:                         1.445e+04
Df Residuals:                   17972   BIC:                         1.465e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3243      0.016     82.329      0.000       1.293       1.356
x1             0.0011      0.000      2.525      0.012       0.000       0.002
x2            -0.0152      0.002     -9.341      0.000      -0.018      -0.012
x3            -0.0004      0.000     -2.205      0.027      -0.001   -4.75e-05
x4            -0.0955      0.006    -16.044      0.000      -0.107      -0.084
x5            -0.1020      0.006    -18.233      0.000      -0.113      -0.091
x6            -0.2382      0.012    -20.055      0.000      -0.261      -0.215
x7            -0.2258      0.008    -26.600      0.000      -0.242      -0.209
x8            -0.1797      0.007    -25.979      0.000      -0.193      -0.166
x9            -0.0601      0.006    -10.606      0.000      -0.071      -0.049
x10           -0.1992      0.007    -30.147      0.000      -0.212      -0.186
x11           -0.2810      0.009    -29.747      0.000      -0.299      -0.262
x12           -0.3266      0.016    -20.123      0.000      -0.358      -0.295
x13           -0.3638      0.017    -21.616      0.000      -0.397      -0.331
x14           -0.2804      0.010    -28.922      0.000      -0.299      -0.261
x15           -0.1587      0.007    -23.615      0.000      -0.172      -0.146
x16           -0.0553      0.005    -10.106      0.000      -0.066      -0.045
x17           -0.1856      0.012    -16.084      0.000      -0.208      -0.163
x18           -0.1221      0.007    -18.257      0.000      -0.135      -0.109
x19           -0.2152      0.009    -24.595      0.000      -0.232      -0.198
x20           -0.1357      0.006    -22.905      0.000      -0.147      -0.124
x21           -0.1633      0.008    -21.764      0.000      -0.178      -0.149
x22           -0.1151      0.006    -18.188      0.000      -0.127      -0.103
x23           -0.0891      0.006    -15.910      0.000      -0.100      -0.078
x24           -0.1302      0.007    -19.036      0.000      -0.144      -0.117
x25           -0.1249      0.008    -15.607      0.000      -0.141      -0.109
==============================================================================
Omnibus:                      283.784   Durbin-Watson:                   1.491
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              297.038
Skew:                           0.314   Prob(JB):                     3.15e-65
Kurtosis:                       3.045   Cond. No.                         269.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

array([[ 1., 23.,  5., ...,  0.,  0.,  0.],
       [ 1., 28.,  3., ...,  0.,  0.,  1.],
       [ 1., 39.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 1., 38.,  1., ...,  1.,  0.,  0.],
       [ 1., 28.,  3., ...,  0.,  0.,  0.],
       [ 1., 36.,  2., ...,  0.,  0.,  0.]])

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25]

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.5, random_state=42)

In [69]:
scaler_cols = ['Age_at_Marriage', 'Children_Count', 'Years_Since_Marriage']
scaler = StandardScaler()

X_train[scaler_cols] = scaler.fit_transform(X_train[scaler_cols])
X_test[scaler_cols] = scaler.transform(X_test[scaler_cols])

display(X_train.head())
display(X_test.head())

,Age_at_Marriage,Children_Count,Years_Since_Marriage,Marriage_Type_Love,Gender_Male,Education_Level_PhD,Education_Level_Postgraduate,Education_Level_School,Caste_Match_Same,Religion_Hindu,...,Urban_Rural_Urban,Dowry_Exchanged_Not Disclosed,Dowry_Exchanged_Yes,Marital_Satisfaction_Low,Marital_Satisfaction_Medium,Income_Level_Low,Income_Level_Middle,Spouse_Working_Yes,Inter_Caste_Yes,Inter_Religion_Yes
1963,0.235896,-0.786822,-0.000111,True,True,False,False,True,False,True,...,True,False,False,True,False,False,False,True,True,False
12124,0.722895,-0.183498,0.071407,False,False,False,False,False,True,True,...,True,False,False,False,False,False,True,False,False,False
14294,-1.712100,-1.390145,-0.143148,False,True,False,False,True,False,False,...,True,False,False,False,True,False,True,True,False,False
17620,1.372227,0.419826,1.072667,False,False,False,False,False,True,False,...,False,False,False,False,True,True,False,True,True,False
13462,-0.575769,-0.786822,-1.430483,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


,Age_at_Marriage,Children_Count,Years_Since_Marriage,Marriage_Type_Love,Gender_Male,Education_Level_PhD,Education_Level_Postgraduate,Education_Level_School,Caste_Match_Same,Religion_Hindu,...,Urban_Rural_Urban,Dowry_Exchanged_Not Disclosed,Dowry_Exchanged_Yes,Marital_Satisfaction_Low,Marital_Satisfaction_Medium,Income_Level_Low,Income_Level_Middle,Spouse_Working_Yes,Inter_Caste_Yes,Inter_Religion_Yes
11364,0.398229,-0.786822,0.071407,False,False,False,False,True,True,False,...,True,False,False,False,True,False,False,False,False,False
4119,-0.900435,0.419826,-0.858334,True,True,True,False,False,True,False,...,True,False,False,False,True,True,False,True,False,False
2008,-1.062768,1.626473,-0.715297,False,True,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,False
12572,-0.575769,-0.786822,1.573297,False,False,False,False,False,True,True,...,True,False,False,False,False,False,True,False,False,False
4755,0.722895,0.419826,1.072667,False,True,False,True,False,True,False,...,True,False,False,False,True,False,True,False,False,True


In [70]:
display(X_train)

,Age_at_Marriage,Children_Count,Years_Since_Marriage,Marriage_Type_Love,Gender_Male,Education_Level_PhD,Education_Level_Postgraduate,Education_Level_School,Caste_Match_Same,Religion_Hindu,...,Urban_Rural_Urban,Dowry_Exchanged_Not Disclosed,Dowry_Exchanged_Yes,Marital_Satisfaction_Low,Marital_Satisfaction_Medium,Income_Level_Low,Income_Level_Middle,Spouse_Working_Yes,Inter_Caste_Yes,Inter_Religion_Yes
1963,0.235896,-0.786822,-0.000111,True,True,False,False,True,False,True,...,True,False,False,True,False,False,False,True,True,False
12124,0.722895,-0.183498,0.071407,False,False,False,False,False,True,True,...,True,False,False,False,False,False,True,False,False,False
14294,-1.712100,-1.390145,-0.143148,False,True,False,False,True,False,False,...,True,False,False,False,True,False,True,True,False,False
17620,1.372227,0.419826,1.072667,False,False,False,False,False,True,False,...,False,False,False,False,True,True,False,True,True,False
13462,-0.575769,-0.786822,-1.430483,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,-0.088770,-1.390145,-0.214667,False,False,False,False,False,True,True,...,True,False,False,False,False,False,False,False,False,False
11964,-0.575769,-1.390145,1.358741,False,True,False,True,False,True,False,...,True,False,False,False,True,False,False,True,False,False
5390,-1.549767,1.023150,1.644816,False,False,False,False,False,True,True,...,True,False,False,True,False,True,False,True,True,False
860,-0.251103,-0.786822,-0.286186,True,False,False,False,False,True,False,...,True,False,False,False,True,False,True,False,False,False


In [ ]:
#Fitting Logistic Regression to Training Set
from sklearn.linear_model import LogisticRegression
classifierObj = LogisticRegression()
classifierObj.fit(X_train, y_train)
#Making predictions on the Test Set
y_pred = classifierObj.predict(X_test)
#Predicting probabilities
y_pred_prob = classifierObj.predict_proba(X_test)
#Print Model Accuracy
print(classifierObj.score(X_test,y_test))

In [ ]:
#Evaluating the predictions using a Confusion Matrix
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
#Print the classification report
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred)
print(report)
# Visualizing the Training set results
from matplotlib.colors import ListedColormap
X_set, y_set = X_train, y_train
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
plt.contourf(X1, X2, classifierObj.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
alpha = 0.75, cmap = ListedColormap(('red', 'blue')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
plt.scatter(X_set[:, 0], X_set[:, 1], c=y_set, cmap=ListedColormap(('red', 'blue')))
plt.scatter(X_set[y_set == 0, 0], X_set[y_set == 0, 1], color = 'red', label = 'Class 0')
plt.scatter(X_set[y_set == 1, 0], X_set[y_set == 1, 1], color = 'blue', label = 'Class 1')
plt.title('Training set')
plt.xlabel('Age')
plt.ylabel('Estimated Salary')
plt.legend()
plt.show()
# Visualizing the Test set results
from matplotlib.colors import ListedColormap
X_set, y_set = X_test, y_test
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                     np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
plt.contourf(X1, X2, classifierObj.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
             alpha = 0.75, cmap = ListedColormap(('red', 'blue')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
plt.scatter(X_set[:, 0], X_set[:, 1], c=y_set, cmap=ListedColormap(('red', 'blue')))
plt.scatter(X_set[y_set == 0, 0], X_set[y_set == 0, 1], color = 'red', label = 'Class 0')
plt.scatter(X_set[y_set == 1, 0], X_set[y_set == 1, 1], color = 'blue', label = 'Class 1')
plt.title('Test set')
plt.xlabel('Age')
plt.ylabel('Estimated Salary')
plt.legend()
plt.show()
